# b_bottom & potential field

In [1]:
from setproctitle import setproctitle
setproctitle("prepare")

import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "2"

In [2]:
import glob 
import numpy as np

from cmspinn.prepare_hmi import PrepareHMI
from cmspinn.potential_field import potential_cube

In [3]:
Nz = 160
batch_size = 2500

hmi_path = '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/'
base_path = os.path.expanduser('~/workspace/workspace_mine/_data/NOAA12673/')

# hmi_path = '/mnt/obsdata/sdo_AR/NOAA11158_HARP377/hmi/'
# base_path = os.path.expanduser('~/workspace/workspace_mine/_data/NOAA11158/')

b_bottom_path = os.path.join(base_path, 'b_bottom')
b_bottom_err_path = os.path.join(base_path, 'b_bottom_err')
b_potential_path = os.path.join(base_path, 'b_potential')
b_potential_energy_path = os.path.join(base_path, 'b_potential_energy')

os.makedirs(b_bottom_path, exist_ok=True)
os.makedirs(b_bottom_err_path, exist_ok=True)
os.makedirs(b_potential_path, exist_ok=True)
os.makedirs(b_potential_energy_path, exist_ok=True)

In [4]:
creater = PrepareHMI(b_bottom_path, hmi_path)
creater.load_hmi_data(idx1=0, idx2=999)

[('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Bp.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Bt.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Br.fits'),
 ('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Bp.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Bt.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Br.fits'),
 ('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Bp.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Bt.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Br.fits'),
 ('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_003600_TAI.Bp.fits

In [5]:
creater.save_bottom(stride=2)

Exist /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_000000.npy


See https://docs.sunpy.org/en/stable/code_ref/map.html#fixing-map-metadata for how to fix metadata before loading it with sunpy.map.Map.
See https://fits.gsfc.nasa.gov/fits_standard.html forthe FITS unit standards. [sunpy.map.mapbase]


Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_001200.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_002400.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_003600.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_004800.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_010000.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_011200.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_012400.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_013600.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_014800.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom

In [6]:
creater = PrepareHMI(b_bottom_err_path, hmi_path)
creater.load_hmi_data(idx1=0, idx2=999)

[('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Bp.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Bt.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_000000_TAI.Br.fits'),
 ('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Bp.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Bt.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_001200_TAI.Br.fits'),
 ('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Bp.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Bt.fits',
  '/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_002400_TAI.Br.fits'),
 ('/mnt/obsdata/sdo_AR/NOAA12673_HARP7115/hmi/hmi.sharp_cea_720s.7115.20170906_003600_TAI.Bp.fits

In [7]:
creater.save_bottom(stride=2)

Exist /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_000000.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_001200.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_002400.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_003600.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_004800.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_010000.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_011200.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_012400.npy
Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom_err/b_bottom_20170906_013600.npy
Saved /userhome/jeon_mg/workspace/wor

In [10]:
b_bottom_list = sorted(glob.glob(os.path.join(b_bottom_path, '*.npy')))
b_bottom_list

['/userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_bottom/b_bottom_20170906_000000.npy']

In [11]:
import pyvista as pv

from cmspinn.evaluation import magnetic_energy

In [12]:
stride = 2
Mm_per_pixel = stride*0.36
cm_per_pixel = Mm_per_pixel*1e8
dV = cm_per_pixel ** 3

for b_bottom_path in b_bottom_list:
    b_bottom_date = os.path.basename(b_bottom_path)[9:-4]
    b_pot_vtk_path = os.path.join(b_potential_path, f'b_potential_{b_bottom_date}.vtk')
    b_pot_me_path = os.path.join(b_potential_energy_path, f'b_me_{b_bottom_date}.npy')
    
    if os.path.exists(b_pot_vtk_path) and os.path.exists(b_pot_me_path):
        print(f'Exist {b_pot_vtk_path} / {b_pot_me_path}')
     
    else:
        with open(b_bottom_path, 'rb') as f:
            b_bottom = np.load(f)

        bz_bottom = b_bottom[:, :, 2]
        potential_maker = potential_cube(bz_bottom, Nz)
        potential_maker.cal_and_save_potential_field(b_pot_vtk_path, batch_size=batch_size)

        Nx, Ny, _ = b_bottom.shape
        b_pot_mesh = pv.read(b_pot_vtk_path)
        b_pot = b_pot_mesh['B'].reshape(Nz, Ny, Nx, 3).transpose(2, 1, 0, 3)
        pot_me = magnetic_energy(b_pot) * dV
        np.save(b_pot_me_path, pot_me)

        print(f'Saved {b_pot_vtk_path} / {b_pot_me_path}')


Potential Field: 100%|██████████| 4932/4932 [03:07<00:00, 26.31it/s]


Saved /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_potential/b_potential_20170906_000000.vtk / /userhome/jeon_mg/workspace/workspace_mine/_data/NOAA12673/b_potential_energy/b_me_20170906_000000.npy
